### all the needed libraries

In [1]:

import matplotlib.pyplot as plt
import cv2
import bz2
import io
from PIL import Image
import numpy as np
import zipfile
import dlib
import pandas as pd
import stone 
from imutils import face_utils
from scipy.spatial import distance
from imutils import face_utils 
import os
import stone
import stone
import colorspacious as cs
from matplotlib import colors as mcolor
from skimage import color,io 

### getting the right files to start

In [2]:
# 
images = os.listdir("/Users/user/Downloads/skin color with computer vision/facesforexperiment2")
folder_path = '/Users/user/Downloads/skin color with computer vision/facesforexperiment2/'
image_list2 = [[f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png'))]]


### initializing the face detetor and facial landmarks

In [3]:
# setting the detector and landmarks for measurements
detector = dlib.get_frontal_face_detector()
landmarks_predictor = dlib.shape_predictor("/Users/user/Downloads/skin color with computer vision/shape_predictor_81_face_landmarks.dat")

### unziping the file 


In [4]:

image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) 
    if os.path.isfile(os.path.join(folder_path, f)) and f.endswith(('.jpg', '.png'))]
image_list2.extend(image_files)

In [5]:
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)


### distance function 

In [6]:
# calculating distance and updating Dataframe

def calculate_distance_and_update_dataframe(landmarks, landmarks1, landmarks2, column_name, idx):
    x1,  y1 = landmarks.part(landmarks1).x, landmarks.part(landmarks1).y
    x2, y2 = landmarks.part(landmarks2).x, landmarks.part(landmarks2).y
    
    distance_value = distance.euclidean((x1,y1), (x2, y2))
    dataframe.at[idx, column_name]= distance_value
    

### using the tip of the nose to divide images for asymmetry comparison

In [7]:



def calculate_facial_symmetry(landmarks, landmark1, landmark2, img):
    h, w = img.shape[:2]
    
    midpoint_x = (landmarks.part(landmark1).x + landmarks.part(landmark2).x) // 2
    
    midpoint_x = min(max(midpoint_x, 0), w)
    
    right_part = img[:,:midpoint_x]
    left_part = img[:, midpoint_x:]
    
    cv2.imshow("right_part",right_part)
    cv2.imshow("left_part",left_part)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    

In [8]:
for image_file in image_files:
    try:
        image_path = os.path.join(folder_path, image_file)
        
        your_image = cv2.imread(image_path)
        
        img = cv2.imread(image_path)
        
        rgb_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        faces = detector(rgb_image)
        
        if faces:
            face = faces[0]
            
            landmarks = landmarks_predictor(rgb_image,face)
            
            calculate_facial_symmetry(landmarks, 27, 8, img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Error: Could not load image at {image_path}")
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
    break

cv2.waitKey(0)
cv2.destroyAllWindows()

 ### uses the stone(skin tone) library to get hex code of skin color detected in photo
### transforms the hex code to rgb, then finally to CIELAB 

In [ ]:
def get_rgb_to_cie(image_path, idx):
    # for idx, image_name in enumerate(image_path):
    
    try:
        current_image = image_path
        output = stone.process(current_image, image_type='color', return_report_image=True)
        
        if output is not None:
            get_output = output.get('faces', [])
            
            if get_output:
                skin_tone = get_output[0].get('skin_tone', None)
                
                if skin_tone:
                    skin_tone_2 = skin_tone.lstrip('#')
                    
                    # Convert skin tone hex to RGB and normalize
                    rgb2 = tuple(int(skin_tone_2[i:i + 2], 16) for i in (0, 2, 4))
                    rgb2 = np.array(rgb2) / 255
                    
                    # Converting RGB to CIE
                    lab = color.rgb2lab(rgb2[np.newaxis, np.newaxis, :])[0][0]
                    lab = np.round(lab, 2)
                    
                    print(f"skin tone: {skin_tone} cielab: {lab}  at {current_image}")
            else:
                print(f"output at {idx} is not found")
        else:
            print(f"image at {idx} failed")
            
    except Exception as e:
        print(f"error processing image {idx}: {e}")

        

### calling the get_rgb_cie function, setting a disired amount of files to test 

In [ ]:
print(image_list2)

In [ ]:
desired_amount = 10
image_count = 0
for idx, image_group in enumerate(image_list2):
    for im in image_group:  # Process each image in the group
        try:
            # Construct full image path
            image_path = os.path.join('/Users/user/Downloads/skin color with computer vision/facesforexperiment2/', im)
            get_rgb_to_cie(image_path, idx)
            image_count += 1
            if image_count >= desired_amount:
                break
            
            
        except Exception as e:
            print(f"error processing image {im}: {str(e)}")

    # Break after first group for testing purposes
    if image_count >= desired_amount:
        break

### dataframe

In [ ]:
#  craniofacial distance and Craniofacial area
dataframe = pd.DataFrame({
"Image Name": [None] * len(images),
'Head Height': [0.0] * len(images),  
'Face Height': [0.0] * len(images),
'Face Height 2': [0.0] * len(images),
'Face Height 3': [0.0] * len(images),
'Face Width': [0.0] * len(images),
'Face Width 2': [0.0] * len(images),
'Orbits Intercanthal Width': [0.0] * len(images),
'Orbits Fissure Length(left and right)': [0.0] * len(images),
'Orbits Biocular Width': [0.0] * len(images),
'Nose Height': [0.0] * len(images),
'Nose Width': [0.0] * len(images),
'Labio-oral region':[0.0] * len(images),
'Intercanthal Face Height': [0.0] * len(images),
'eye fissure height (left and right)': [0.0] * len(images),
'orbit and brow height (left and right)': [0.0] * len(images),
'Columella Length': [0.0] * len(images),
'Upper Lip Height': [0.0] * len(images),
'Lower Vermilion Height': [0.0] * len(images),
'Philtrum Width': [0.0] * len(images),
'lateral upper lip heights (left and right)': [0.0] * len(images),
'Lower Face Height': [0.0] * len(images),
'Upper Vermilion Height': [0.0] * len(images)

})

### calculating measurements

In [ ]:
import cv2
import numpy as np

# Define the measurement columns once outside the loop
measurement_columns = [
    'Head Height', 'Face Height', 'Face Height 2', 'Face Height 3', 'Face Width', 'Face Width 2',
    'Orbits Intercanthal Width', 'Orbits Fissure Length(left and right)', 'Orbits Biocular Width',
    'Nose Height', 'Nose Width', 'Labio-oral region', 'Intercanthal Face Height',
    'eye fissure height (left and right)', 'orbit and brow height (left and right)', 'Columella Length',
    'Upper Lip Height', 'Lower Vermilion Height', 'Philtrum Width', 'lateral upper lip heights (left and right)',
    'Lower Face Height','Upper Vermilion Height'
]


dataframe = pd.DataFrame({col: [np.nan] * len(images) for col in measurement_columns})
dataframe['Image Name'] = [None] * len(images)

for idx, im in enumerate(images):
    try:
        # Skip DS_Store
        if im == "DS_Store":
            continue
        
        # Read and check the image
        image_path = '/Users/user/Downloads/skin color with computer vision/facesforexperiment2/' + im
        image = cv2.imread(image_path)
        
        # Check if image loaded successfully
        if image is None:
            print(f"Unable to read image {im}")
            dataframe.loc[idx, measurement_columns] = np.nan  
            continue  

        # Store the image name in the dataframe
        dataframe.at[idx, "Image Name"] = im
        
        # Convert the image to RGB color space and detect faces
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        faces = detector(rgb_image)
        
        # Ensure that at least one face is detected before proceeding
        if len(faces) == 0:
            print(f"No face detected in image {im}")
            dataframe.loc[idx, measurement_columns] = np.nan 
            continue
        
        # Get the first face detected
        face = faces[0]
        landmarks = landmarks_predictor(rgb_image, face)
        
        # Perform distance calculations and update the DataFrame
        calculate_distance_and_update_dataframe(landmarks, 71, 27, 'Head Height', idx)
        calculate_distance_and_update_dataframe(landmarks, 71, 8, 'Face Height', idx)
        calculate_distance_and_update_dataframe(landmarks, 27, 8, 'Face Height 2', idx)
        calculate_distance_and_update_dataframe(landmarks, 33, 8, 'Face Height 3', idx)
        calculate_distance_and_update_dataframe(landmarks, 2, 14, 'Face Width', idx)
        calculate_distance_and_update_dataframe(landmarks, 5, 11, 'Face Width 2', idx)
        calculate_distance_and_update_dataframe(landmarks, 39, 42, 'Orbits Intercanthal Width', idx)
        calculate_distance_and_update_dataframe(landmarks, 39, 36, 'Orbits Fissure Length(left and right)', idx)
        calculate_distance_and_update_dataframe(landmarks, 36, 45, 'Orbits Biocular Width', idx)
        calculate_distance_and_update_dataframe(landmarks, 27, 33, 'Nose Height', idx)
        calculate_distance_and_update_dataframe(landmarks, 31, 35, 'Nose Width', idx)
        calculate_distance_and_update_dataframe(landmarks, 48, 54, 'Labio-oral region', idx)
        calculate_distance_and_update_dataframe(landmarks, 27, 66, 'Intercanthal Face Height', idx)
        calculate_distance_and_update_dataframe(landmarks, 38, 41, 'eye fissure height (left and right)', idx)
        calculate_distance_and_update_dataframe(landmarks, 19, 41, 'orbit and brow height (left and right)', idx)
        calculate_distance_and_update_dataframe(landmarks, 33, 30, 'Columella Length', idx)
        calculate_distance_and_update_dataframe(landmarks, 33, 66, 'Upper Lip Height', idx)
        calculate_distance_and_update_dataframe(landmarks, 66, 57, 'Lower Vermilion Height', idx)
        calculate_distance_and_update_dataframe(landmarks, 49, 53, 'Philtrum Width', idx)
        calculate_distance_and_update_dataframe(landmarks, 32, 62, 'lateral upper lip heights (left and right)', idx)
        # distance between stomion and gnathion
        calculate_distance_and_update_dataframe(landmarks, 66, 8, 'Lower Face Height', idx)
        # distance between stomion and libiale superious
        calculate_distance_and_update_dataframe(landmarks, 62, 66, 'Upper Vermilion Height', idx)
        
    except Exception as e:
        print(f"Error processing image {im}: {str(e)}")
        dataframe.loc[idx, measurement_columns] = np.nan 
        continue  # Continue to the next image in case of an error

# Show the DataFrame
pd.set_option('display.max_columns', None)


In [ ]:
dataframe['Facial index'] = dataframe['Face Height 2']/dataframe['Face Width']
dataframe['Mandibular index'] = dataframe['Lower Face Height']/dataframe['Face Width 2']
dataframe['Intercanthal index'] = dataframe['Orbits Intercanthal Width']/dataframe['Orbits Biocular Width']
dataframe['Orbital width index (left and right)'] = dataframe['Orbits Fissure Length(left and right)']/dataframe['Orbits Intercanthal Width']
dataframe['Eye fissure index (left and right)'] = dataframe['eye fissure height (left and right)']/dataframe['Orbits Fissure Length(left and right)']
dataframe['Nasal index'] = dataframe['Nose Width']/dataframe['Nose Height']
dataframe['Vermilion height index '] = dataframe['Upper Vermilion Height']/dataframe['Lower Vermilion Height']
dataframe['Mouth-face width index'] = dataframe['Labio-oral region']/dataframe['Face Width']

pd.set_option('display.max_columns', None)

### Individual Typology Angle

In [ ]:
average_ita_values = []


for image_file in image_files:
    
    image_path = os.path.join(folder_path, image_file)

    your_image = cv2.imread(image_path)
    # Resize the image to reduce memory usage
    your_image = cv2.resize(your_image, (500, 500))  # Resize to 500x500 pixels

    
    lab_img = color.rgb2lab(your_image.astype('uint8'))

    
    ita_values = []

    for pixel in lab_img.reshape((-1, 3)):
        L, a, b = pixel

        if L > 0:  
            ita = np.arctan((L - 50) / b) * (180 / np.pi)
            ita_values.append(ita)

    average_ita = np.mean(ita_values)


    average_ita_values.append(average_ita)


# Check if lengths match before adding to dataframe
if len(average_ita_values) < len(dataframe):
    average_ita_values.extend([np.nan] * (len(dataframe) - len(average_ita_values)))
elif len(average_ita_values) > len(dataframe):
    average_ita_values = average_ita_values[:len(dataframe)]

dataframe['Average_ITA'] = average_ita_values






In [ ]:
print(dataframe)